In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.database import database
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
# fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = database.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0         0.0     2.0     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1         1.0     2.0     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2         2.0     2.0     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3         3.0     2.0     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15           6.0                  NaN      1.0         1.0  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4         4.0     2.

In [13]:
#####################################
########## SHOW 3D PLOTS ############
#####################################


#print (len(cohort.start_times_military), cohort.start_times_military[0])
print (len(cohort.start_times_absolute_minute))

#
cohort.animal_id = 5
cohort.exclude_huddles = True
cohort.subsample_rate = 24
cohort.pdays = ["p15", "p20", "P25","P30"] 

#
cohort.show_3D_plots()


991
pday starts:  [0, 7200, 14400, 21600]


 13%|█████▏                                 | 198/1500 [00:00<00:02, 445.70it/s]

Error loading track:  54
Error loading track:  61


100%|██████████████████████████████████████| 1500/1500 [00:07<00:00, 204.03it/s]


In [9]:
#####################################
########## SHOW ETHOGRAMS ###########
#####################################

cohort.animal_ids = [0,1,2,3,4,5]

all_pairs = [(a, b) for idx, a in enumerate(animal_ids) for b in animal_ids[idx + 1:]]
print ("all_pairs: ", all_pairs)

#
for pair in all_pairs:

    cohort.animal_ids = [pair[0],pair[1]]
    
    cohort.exclude_huddles = True
    ethogram.plot_ethogram_hourly(n_partitions,
                                 behavior_name,
                                 cohort,
                                 cohort.exclude_huddles,
                                 vmax=50)


all_pairs:  [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]


In [70]:
################################################
########### CREATE ROI OF INTEREST #############
################################################
# need to select 1 video and 1 slp for visualization purposes;
# make sure the video is shirnk cropped and matches the traces from the slop also.
cohort.fname_video ='/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/2020_07_26_00_05_52_835502_compressed_defished_shrink_cropped.mp4'
cohort.fname_slp_npy =  '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy'

#
cohort.load_video2()

#
cohort.set_roi()

print ("rectangular coords: ", cohort.rect_coords)
print ("circular coords: ", cohort.circle_coords)


ASSUMING VIDEO IS 700 x 900... this only works for cohort2...
(28802, 6, 2)
rectangular coords:  [[0. 0.]
 [0. 0.]]
circular coords:  [[170.56699752 487.73343445]
 [247.39081886 558.83022087]]


In [59]:
################################################
########### COMPUTE CIRCLE OCCUPANCY ###########
################################################

#
behavior_name = 'circular_region'

# select animals
cohort.exclude_huddles = True

# select animals
animal_ids = [0,1,2,3,4,5]

#
for animal_id in animal_ids:

    #
    #cohort.animals = animal_id
    cohort.animal_ids = animal_id

    #
    cohort.get_circle_occupancy(cohort.animal_ids)

    #
    cohort.format_behavior()
    
    #
    ethogram.generate_ethogram_hourly(n_partitions, 
                                      behavior_name,
                                      cohort,
                                      cohort.exclude_huddles)

100%|██████████| 1500/1500 [00:00<00:00, 4617.35it/s]


In [64]:
################################################
########### COMPUTE RECTANGLE OCCUPANCY ########
################################################

#
behavior_name = 'rectangular_region'

# select animals
cohort.exclude_huddles = True

# select animals
animal_ids = [0,1,2,3,4,5]

#
for animal_id in animal_ids:

    #
    cohort.animal_ids = animal_id

    #
    cohort.get_rectangle_occupancy(cohort.animal_ids)

    #
    cohort.format_behavior()
    
    #
    ethogram.generate_ethogram_hourly(n_partitions, 
                                      behavior_name,
                                      cohort,
                                      cohort.exclude_huddles)

100%|██████████| 1500/1500 [00:00<00:00, 5187.88it/s]


res:  (1500,)


100%|██████████| 1500/1500 [00:00<00:00, 5652.57it/s]


res:  (1500,)


100%|██████████| 1500/1500 [00:00<00:00, 5699.60it/s]


...

res:  (1500,)


100%|██████████| 1500/1500 [00:00<00:00, 5590.20it/s]


res:  (1500,)


100%|██████████| 1500/1500 [00:00<00:00, 5738.32it/s]

res:  (1500,)


In [10]:
######################################
########### PCA SCATTER PLOTS ########
######################################

#
animal_ids = [0,1,2,3,4,5]

# choose one of these behaviors
# cohort.behavior_name = 'rectangular_region'
#cohort.behavior_name = 'circular_region'
cohort.behavior_name = 'pairwise_proximity'

# for single animal behaviors
if cohort.behavior_name=='pairwise_proximity':
    cohort.animal_ids = list((a, b) for idx, a in enumerate(animal_ids) for b in animal_ids[idx + 1:])
else:
    cohort.animal_ids = animal_ids

#
cohort.remove_zeros=True
cohort.show_developmental_trajectories()

#


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]
(240, 24)
sums:  7639.942249380831
X_pca:  (240, 3)
removing zeros:  (0,)


In [16]:
#######################################################
########### REMOVE HUDDLES FROM FEATURE TRACKS ########
#######################################################

#
track1 = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine.npy')
track2 = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/features/2020_07_26_00_05_52_835502_compressed_Night_spine_nohuddle.npy')

#
huddle = np.load('/media/cat/4TBSSD/dan/cohort2/P21_P28/huddles/2020_07_26_00_05_52_835502_compressed_Night_huddle_spine_fixed_interpolated.npy')
print (huddle.shape)

print (track1.shape)
animal_id = 0

# 
plt.figure()
# plt.plot(track1[:,animal_id,0],
#         track1[:,animal_id,1],
#         c='red',
#         alpha=.5)
# 
plt.plot(track2[:,animal_id,0],
        track2[:,animal_id,1]+10,
        c='blue',
        alpha=.5)
# 
plt.scatter(huddle[:,0,0], huddle[:,0,1],
           s=50,
           alpha=.5)
plt.xlim(0,900)
plt.ylim(0,700)
plt.show()

# 




(28802, 1, 2)
(28802, 6, 2)
